In [31]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.lines as mlines
import os
from IPython.display import display
import networkx as nx
import statistics
import scipy.stats as st
from sklearn.metrics import jaccard_score
from haversine import haversine
import math
from graphviz import Digraph

In [2]:
df_martix = pd.read_csv('./study_data/cities_gzh_aja_martix.csv')
df_citys = pd.read_csv('./study_data/sa340.csv')
df_martix.set_index('city', inplace=True)
rank = pd.read_csv('./study_data/score/城市信息+排名得分综合表.csv')
df_in_out = pd.read_csv('./study_data/all_cities_in_out_count.csv')

In [64]:
def city_in_out_degree(city):
    # /////////STEP1：处理边//////////
    # out
    temp_out = df_martix.loc[[city]]
    datas_out = []
    for column, value in zip(temp_out.columns, temp_out.iloc[0]):
        if value > 0:
            datas_out.append([city, column, value, 'out'])
    df_out = pd.DataFrame(datas_out, columns=['source_Label', 'target_Label', 'weight', '_class'])
    
    # in 
    temp_in = df_martix[city]
    datas_in = []
    for index, value in temp_in.items():
        if value > 0:
            datas_in.append([index, city, value, 'in'])
    df_in = pd.DataFrame(datas_in, columns=['source_Label', 'target_Label', 'weight', '_class'])
    
    # 合并 in + out, 组成边
    edges = pd.concat([df_in, df_out])
    
    # /////////STEP2：处理节点编号//////////
    # 将节点编号为ID
    city_names = list(set(edges['source_Label'].tolist() + edges['target_Label'].tolist()))
    city_dict = {city: i for i, city in enumerate(city_names)}
    
    # /////////STEP3：给edges增加城市编号//////////
    edges_data = []
    for i, r in edges.iterrows():
        source_Label = r['source_Label']
        target_Label = r['target_Label']
        source_id = city_dict.get(source_Label)
        target_id = city_dict.get(target_Label)
        # 不需要加权重r['Weight'],
        edges_data.append([source_id, target_id, source_Label, target_Label,  r['_class'] ])
        
    final_edges = pd.DataFrame(edges_data, columns=['source', 'target', 'source_Label', 'target_Label', '_class'])
        
    #//////STEP4: 处理节点///////
    nodes_data = []
    for i in city_names:
        city_id = city_dict.get(i)
        label = df_citys[df_citys.name==i]['en_name'].values[0]
        nodes_data.append([city_id, i, label])
    final_nodes = pd.DataFrame(nodes_data, columns=['Id', 'name','label'])
    
    #////STEP5:保存数据////
    folder_path = './地图_QGIS/top10_last10/' + city + '/'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"文件夹 '{folder_path}' 已创建。")
        
    final_edges.to_csv(folder_path + 'edges.csv', index=None)
    final_nodes.to_csv(folder_path + 'nodes.csv', index=None)
    
    #////TODO: STEP6: gephi可视化好看/////
    
    return final_nodes, final_edges